In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta

In [2]:
#Open excel workbook and first sheet
xls = pd.ExcelFile("C:/Users/jason/D3-Visaulization/raw_data/Brent_oil_price.xls")
df = pd.read_excel(xls, "Data 1", skiprows=2)
df.head()

,Date,Weekly Europe Brent Spot Price FOB (Dollars per Barrel)
0,1987-05-15,18.58
1,1987-05-22,18.54
2,1987-05-29,18.60
3,1987-06-05,18.70
4,1987-06-12,18.75


In [3]:
#We only want dates that correspond to viral outbreak dates
#Start with SARS
sars_df = pd.read_csv("C:/Users/jason/D3-Visaulization/cleaned_data/In_progress/SARS_data.csv")
sars_df.head()

,Date,Country,Cases,Deaths
0,2003-03-17,Germany,1,0
1,2003-03-17,Canada,8,2
2,2003-03-17,Singapore,20,0
3,2003-03-17,Hong Kong,95,1
4,2003-03-17,Switzerland,2,0


In [4]:
#Datetime64 and object could not merge; make both datetime 
#-- from stackoverflow.com/questions/55974994/typeerror-dtype-class-datetime-timedelta-not-understood
df["Date"] = pd.to_datetime(df['Date'], errors='coerce')
sars_df["Date"] = pd.to_datetime(sars_df['Date'], errors='coerce')

In [5]:
#Merge the dataframes, then drop unwanted rows to get only the relevant dates
oil_sars = pd.merge(df, sars_df, how="right", on=["Date", "Date"])
del oil_sars["Country"]
#oil_sars.dropna("Weekly Europe Brent Spot Price FOB (Dollars per Barrel)")
oil_sars.head()

,Date,Weekly Europe Brent Spot Price FOB (Dollars per Barrel),Cases,Deaths
0,2003-03-21,28.18,9,2
1,2003-03-21,28.18,0,0
2,2003-03-21,28.18,1,0
3,2003-03-21,28.18,203,6
4,2003-03-21,28.18,1,0


In [20]:
#Groupby date, sum cases and deaths for each date
sars_cases = oil_sars.groupby(["Date"]).sum()["Cases"]
sars_deaths = oil_sars.groupby(["Date"]).sum()["Deaths"]

In [13]:
#Next do MERS
mers_df = pd.read_csv("C:/Users/jason/D3-Visaulization/cleaned_data/In_progress/mers_final.csv")
mers_df.head()

,Unnamed: 0,Year,Week,day,Country,Cases,Continent,lat,long
0,0,2015,18,0,SA,7,Asia,23.885942,45.079162
1,1,2015,22,30,SA,6,Asia,23.885942,45.079162
2,2,2015,26,60,SA,1,Asia,23.885942,45.079162
3,3,2015,30,90,SA,14,Asia,23.885942,45.079162
4,4,2015,18,0,KR,0,Asia,35.907757,127.766922


In [14]:
#Drop unnecessary columns
del mers_df["Unnamed: 0"]
del mers_df["Continent"]
del mers_df["lat"]
del mers_df["long"]
mers_df

,Year,Week,day,Country,Cases
0,2015,18,0,SA,7
1,2015,22,30,SA,6
2,2015,26,60,SA,1
3,2015,30,90,SA,14
4,2015,18,0,KR,0
5,2015,22,30,KR,86
6,2015,26,60,KR,3
7,2015,30,90,KR,0


In [21]:
#Separate Year, Week, Day columns need to be translated to datetime
#From https://www.epochconverter.com/weeks/2015: 
#week 18=2015-04-27, week 22=2015-05-25, week 26=2015-06-22, week 30=2015-07-20
Date = []
for row in range(len(mers_df)):
    if "Week" == 18:
        Date.append("2015-4-27")
    elif "Week" == 22:
        Date.append("2015-5-25")
    elif "Week" == 26:
        Date.append("2015-6-22")
    else:
        Date.append("2015-7-20")

#mers_df["Date"] = Date
#mers_df
print(Date)

['2015-7-20', '2015-7-20', '2015-7-20', '2015-7-20', '2015-7-20', '2015-7-20', '2015-7-20', '2015-7-20']


In [16]:
#COVID-19 is next
covid_df = pd.read_csv("C:/Users/jason/D3-Visaulization/cleaned_data/In_progress/covid_daily_world.csv")
covid_df.head()

,Date,Country/Region,Confirmed,Deaths
0,2020-01-22,Hong Kong,0,0
1,2020-01-22,Japan,2,0
2,2020-01-22,Macao,1,0
3,2020-01-22,China,547,17
4,2020-01-22,South Korea,1,0


In [17]:
#Merge with oil
covid_df["Date"] = pd.to_datetime(covid_df['Date'], errors='coerce')
oil_covid = pd.merge(df, covid_df, how="right", on=["Date", "Date"])
del oil_covid["Country/Region"]
#oil_sars.dropna("Weekly Europe Brent Spot Price FOB (Dollars per Barrel)")
oil_covid.head()

,Date,Weekly Europe Brent Spot Price FOB (Dollars per Barrel),Confirmed,Deaths
0,2020-01-24,62.2,2,0
1,2020-01-24,62.2,2,0
2,2020-01-24,62.2,2,0
3,2020-01-24,62.2,2,0
4,2020-01-24,62.2,916,26


In [22]:
#Groupby date, sum cases and deaths for each date
covid_cases = oil_covid.groupby(["Date"]).sum()["Confirmed"]
covid_deaths = oil_covid.groupby(["Date"]).sum()["Deaths"]

In [18]:
#And finally, ebola
ebola_df = pd.read_csv("C:/Users/jason/D3-Visaulization/cleaned_data/In_progress/ebola_final.csv")
ebola_df.head()

,Unnamed: 0,Country,Date,No. of confirmed cases,No. of confirmed deaths,datetime,continent,code
0,0,Guinea,2014-08-29,482.0,287.0,2014-08-29,Africa,GQ
1,1,Nigeria,2014-08-29,15.0,6.0,2014-08-29,Africa,NG
2,2,Sierra Leone,2014-08-29,935.0,380.0,2014-08-29,Africa,SL
3,3,Liberia,2014-08-29,322.0,225.0,2014-08-29,Africa,LR
4,4,Sierra Leone,2014-09-05,1146.0,443.0,2014-09-05,Africa,SL


In [19]:
#Merge with oil
ebola_df["Date"] = pd.to_datetime(covid_df['Date'], errors='coerce')
oil_ebola = pd.merge(df, ebola_df, how="right", on=["Date", "Date"])
del oil_ebola["Unnamed: 0"]
del oil_ebola["Country"]
del oil_ebola["datetime"]
del oil_ebola["continent"]
del oil_ebola["code"]
#oil_sars.dropna("Weekly Europe Brent Spot Price FOB (Dollars per Barrel)")
oil_ebola.head()

,Date,Weekly Europe Brent Spot Price FOB (Dollars per Barrel),No. of confirmed cases,No. of confirmed deaths
0,2020-01-24,62.2,1513.0,517.0
1,2020-01-24,62.2,812.0,631.0
2,2020-01-24,62.2,1.0,0.0
3,2020-01-24,62.2,19.0,7.0
4,2020-01-24,62.2,1640.0,545.0


In [23]:
#Groupby date, sum cases and deaths for each date
ebola_cases = oil_ebola.groupby(["Date"]).sum()["No. of confirmed cases"]
ebola_deaths = oil_ebola.groupby(["Date"]).sum()["No. of confirmed deaths"]